In [1]:
import requests
import pandas as pd
import datetime as dt
import time
import psycopg2

In [2]:
conn = psycopg2.connect(dbname='reddit-db', user="postgres", host='localhost', password='password', port='5432')
cur = conn.cursor()

In [3]:
api = "https://api.pushshift.io/reddit/search/submission"
#4/1/2015 00:00:00
after_time = 1427860800
#1/1/2018 00:00:00
final_time = 1514782800
#response limit
response_limit = 500

In [4]:
top_sus_subreddit_sql = """select subreddit, count(*) from(
select *
from (
select author, title as submission, created_utm as time_created, subreddit, 'post' as submission_type
from public.sus_user_posts
where title !='This account is banned and is temporarily preserved for purposes of transparency.') as posts_aggregate
union
select *
from (
select author, comment_body as submission, created_utc as time_created, subreddit, 'comment' as submission_type
from public.sus_user_comments) as comments_aggregate) as total_events
group by subreddit
order by count(*) desc"""

In [5]:
subreddits = []
cur.execute(top_sus_subreddit_sql)
for row in cur:
    subreddits.append(row[0])

In [6]:
# for subreddit in subreddits:
#     counter = after_time
#     printcount = 0 
#     while counter < final_time:
#         try:
#             request = requests.get(f'{api}?subreddit={subreddit}&after={counter}&size=500')
#             response = request.json()
#             result = response["data"]
#             counter_list = []
#             for i in result:
#                 cur.execute(f'INSERT INTO public.posts (title, score, id, url, comms_num, created, author, author_flair_text, permalink) values (%s, %s, %s, %s, %s, %s, %s, %s, %s)', (i["title"], i["score"], i["id"], i["url"], i["num_comments"], i["created_utc"], i["author"], i["author_flair_text"], i["permalink"]))
#                 counter_list.append(i["created_utc"])
#             counter = max(counter_list)
#             printcount += 1
#             conn.commit()
#             print(f'Success: Req #{printcount}. -- date={get_date(counter)} -- Working on {subreddit}')
#         except:
#             print(f'ERROR: Req#{printcount} -- date={get_date(counter)} ---- {counter} -- Working on {subreddit}')
#             time.sleep(1)

In [7]:
insert_sql = """ insert into public.normal_user_seed_posts (seed_posts_author, subreddit) values (%s, %s)"""

In [8]:
def check_if_sub_is_alive(subreddit, final_time):
    try:
        time_request = requests.get(f'{api}?subreddit={subreddit}&size=1')
        time_response = time_request.json()
        max_time = time_response["data"][0]["created_utc"]
        if max_time > final_time:
            return final_time
        else:
            return max_time
    except Exception as e:
        print(f'api pull error --{e}')
        print(time_request.status_code)
        print("hit an error checking sub")
        time.sleep(1)
        return check_if_sub_is_alive(subreddit, final_time)

In [9]:
for sub in subreddits[126:]:
    counter = after_time
    printcount = 0
    post_count = 0
    stop_time = check_if_sub_is_alive(sub, final_time)
    while counter < stop_time and post_count < 1000:
        try:
            request = requests.get(f'{api}?subreddit={sub}&after={counter}&size={response_limit}')
            response = request.json()
            result = response["data"]
            counter_list = []
            for i in result:
                cur.execute(insert_sql, (i["author"], i["subreddit"]))
                counter_list.append(i["created_utc"])
                post_count +=1
            conn.commit()
            counter = max(counter_list)
            time.sleep(1.001)
        except Exception as e:
            time.sleep(1)
            print(f'api pull error --{e}')
            print(f'{i["subreddit"]} -- {request.status_code}')

api pull error --list index out of range
200
hit an error checking sub
api pull error --list index out of range
200
hit an error checking sub
api pull error --list index out of range
200
hit an error checking sub
api pull error --list index out of range
200
hit an error checking sub
api pull error --list index out of range
200
hit an error checking sub
api pull error --list index out of range
200
hit an error checking sub
api pull error --list index out of range
200
hit an error checking sub
api pull error --list index out of range
200
hit an error checking sub
api pull error --list index out of range
200
hit an error checking sub
api pull error --list index out of range
200
hit an error checking sub
api pull error --list index out of range
200
hit an error checking sub
api pull error --list index out of range
200
hit an error checking sub
api pull error --list index out of range
200
hit an error checking sub
api pull error --list index out of range
200
hit an error checking sub
api pu

Error: Canceled future for execute_request message before replies were done